Proof:
    - hypothesis
    - axiom (provide values for a, b, c, no need for them to be proven)
    - MP (provide both parts, must be proven)

In [1]:
from collections import OrderedDict

In [2]:
class Proof(list):
    def __repr__(self):
        def repr_proof_step(step):
            if isinstance(step, str):
                return step
            elif isinstance(step, tuple):
                mp, p1, p2 = step
                assert mp == 'mp'
                return 'Application of MP to B{} and B{}'.format(p1, p2)
        
        return '\n'.join(
            'B{}: {} \n    {}'.format(i, st, repr_proof_step(proof_step))
                for i, st, proof_step in self
        )

In [3]:
class Statement:
    @property
    def proof(self):
        statements = OrderedDict()
        
        def visit_statement(statement):
            if statement in statements:
                return statements[statement][0]

            if isinstance(statement.step_proof, tuple):
                (mp, s1, s2) = statement.step_proof
                assert mp == 'mp'
                p1 = visit_statement(s1)
                p2 = visit_statement(s2)
                step_proof = ('mp', p1, p2)
            else:
                step_proof = statement.step_proof
            
            num = len(statements)
            statements[statement] = (num, step_proof)
            return num

        visit_statement(self)
        return Proof((i, st, pf) for (st, (i, pf)) in statements.items())

In [4]:
class Implication(Statement):
    def __init__(self, antecedent, consequent, proof=None):
        self.antecedent = antecedent
        self.consequent = consequent
        self.step_proof = proof
    
    def __repr__(self):
        return '({} → {})'.format(self.antecedent, self.consequent)
    
    def __eq__(self, other):
        return (self.antecedent == other.antecedent) and (self.consequent == other.consequent)
    
    def __hash__(self):
        return hash((self.antecedent, self.consequent))
    
    def __iter__(self):
        yield self.antecedent
        yield self.consequent

In [5]:
class Symbol(Statement):
    def __init__(self, name, proof=None):
        self.name = name
        self.step_proof = proof
        
    def __repr__(self):
        return self.name

In [6]:
i = Implication
s = Symbol

In [7]:
def a1(a, b):
    return i(a, i(b, a), 'axiom A1')

In [8]:
def a2(a, b, c):
    return i(i(a, i(b, c)), i(i(a, b), i(a, c)), 'axiom A2')

In [9]:
def mp(p1, p2):
    assert p2.antecedent == p1, 'needed {} but got {}'.format(p2.antecedent, p1)
    p2.consequent.step_proof = ('mp', p1, p2)
    return p2.consequent

In [37]:
A,B,C='🐹🍊🍷'

In [38]:
P1 = i(A, B, 'h1')
P2 = i(B, C, 'h2')
P3 = a2(A, B, C)
P4 = a1(i(B, C), A)
P5 = mp(P2, P4)
P6 = mp(P5, P3)
P7 = mp(P1, P6)

In [39]:
P1 = s(A, 'h1')
P2 = a1(P1, P1)
P3 = mp(P1, P2)

In [40]:
P1 = i(A, B, 'h1')
P2 = i(B, C, 'h2')
P3 = a2(A, B, C)
P4 = a1(i(B,C), A)
P5 = mp(P2, P4)
P6 = mp(P5, P3)
P7 = mp(P1, P6)

In [41]:
P7.proof

B0: (🐹 → 🍊) 
    h1
B1: (🍊 → 🍷) 
    h2
B2: ((🍊 → 🍷) → (🐹 → (🍊 → 🍷))) 
    axiom A1
B3: (🐹 → (🍊 → 🍷)) 
    Application of MP to B1 and B2
B4: ((🐹 → (🍊 → 🍷)) → ((🐹 → 🍊) → (🐹 → 🍷))) 
    axiom A2
B5: ((🐹 → 🍊) → (🐹 → 🍷)) 
    Application of MP to B3 and B4
B6: (🐹 → 🍷) 
    Application of MP to B0 and B5

In [42]:
def deduction(a_implies_b, b_implies_c):
    a, b = a_implies_b
    b1, c = b_implies_c
    assert b == b1
    
    P3 = a2(a, b, c)
    P4 = a1(i(b, c), a)
    P5 = mp(b_implies_c, P4)
    P6 = mp(P5, P3)
    P7 = mp(a_implies_b, P6)
    return P7

In [43]:
deduction(i(C, A, 'h1'), i(A, B, 'h2')).proof

B0: (🍷 → 🐹) 
    h1
B1: (🐹 → 🍊) 
    h2
B2: ((🐹 → 🍊) → (🍷 → (🐹 → 🍊))) 
    axiom A1
B3: (🍷 → (🐹 → 🍊)) 
    Application of MP to B1 and B2
B4: ((🍷 → (🐹 → 🍊)) → ((🍷 → 🐹) → (🍷 → 🍊))) 
    axiom A2
B5: ((🍷 → 🐹) → (🍷 → 🍊)) 
    Application of MP to B3 and B4
B6: (🍷 → 🍊) 
    Application of MP to B0 and B5

In [44]:
P0 = i(A, i(B, C), 'hypothesis')
P1 = a2(A, B, C)
P2 = mp(P0, P1)
P3 = a1(B, A)
P4 = deduction(P3, P2)

In [46]:
P4

(🍊 → (🐹 → 🍷))

In [45]:
P4.proof

B0: (🍊 → (🐹 → 🍊)) 
    axiom A1
B1: (🐹 → (🍊 → 🍷)) 
    hypothesis
B2: ((🐹 → (🍊 → 🍷)) → ((🐹 → 🍊) → (🐹 → 🍷))) 
    axiom A2
B3: ((🐹 → 🍊) → (🐹 → 🍷)) 
    Application of MP to B1 and B2
B4: (((🐹 → 🍊) → (🐹 → 🍷)) → (🍊 → ((🐹 → 🍊) → (🐹 → 🍷)))) 
    axiom A1
B5: (🍊 → ((🐹 → 🍊) → (🐹 → 🍷))) 
    Application of MP to B3 and B4
B6: ((🍊 → ((🐹 → 🍊) → (🐹 → 🍷))) → ((🍊 → (🐹 → 🍊)) → (🍊 → (🐹 → 🍷)))) 
    axiom A2
B7: ((🍊 → (🐹 → 🍊)) → (🍊 → (🐹 → 🍷))) 
    Application of MP to B5 and B6
B8: (🍊 → (🐹 → 🍷)) 
    Application of MP to B0 and B7